# Project 1: Fire Extinguisher

James Burns & Remington Katz

CS 520

In [78]:
import random
import copy

### 1 The Ship

In [79]:
class ship():
    def __init__(self, d):
        self.dim = d
        self.ship_display = [['■' for i in range(d)] for j in range(d)] #■ for blocked i.e. not open
        self.start = ()

        #Choose a square in the interior to open at random
        y = random.randint(1, d-2)
        x = random.randint(1, d-2)
        self.ship_display[y][x] = 'O' #O for open
        #The board display can be imagined as Quadrant IV of a cartesian coordinate system, where the origin is in the top left and you read a coordinate as y spaces down and x spaces to the right
        #Iteratively do the following... (from assignment)
        while True:
            one_open_neighbor = list()
            for row in range(d):
                for col in range(d):
                    if self.ship_display[row][col] == '■':
                        open_neighbor_count = 0
                        if (row+1 <= d-1) and (self.ship_display[row+1][col] == 'O'):
                            open_neighbor_count += 1
                        if (row-1 >= 0) and (self.ship_display[row-1][col] == 'O'):
                            open_neighbor_count += 1
                        if (col+1 <= d-1) and (self.ship_display[row][col+1] == 'O'):
                            open_neighbor_count += 1
                        if (col-1 >= 0) and (self.ship_display[row][col-1] == 'O'):
                            open_neighbor_count += 1
                        if open_neighbor_count == 1:
                            one_open_neighbor.append((row,col))
            if len(one_open_neighbor) == 0:
                break
            random_neighbor = random.sample(one_open_neighbor, 1)[0]
            self.ship_display[random_neighbor[0]][random_neighbor[1]] = 'O'
        #ship.print_ship(self)
        
        #Open dead ends; pick random closed neighbor and open it
        dead_ends_list = list()
        for row in range(d):
            for col in range(d):
                if self.ship_display[row][col] == 'O':
                    open_neighbor_count = 0
                    if (row+1 <= d-1) and (self.ship_display[row+1][col] == 'O'):
                        open_neighbor_count += 1
                    if (row-1 >= 0) and (self.ship_display[row-1][col] == 'O'):
                        open_neighbor_count += 1
                    if (col+1 <= d-1) and (self.ship_display[row][col+1] == 'O'):
                        open_neighbor_count += 1
                    if (col-1 >= 0) and (self.ship_display[row][col-1] == 'O'):
                        open_neighbor_count += 1
                    if open_neighbor_count == 1:
                        dead_ends_list.append((row,col))
        half = len(dead_ends_list)//2
        dead_ends = random.sample(dead_ends_list, half)
        #print(dead_ends_list)
        #print(dead_ends)
        for dead_end in dead_ends:
            dead_end_blocked_neighbor = list()
            if (dead_end[0]+1 <= d-1) and (self.ship_display[dead_end[0]+1][dead_end[1]] == '■'):
                dead_end_blocked_neighbor.append((dead_end[0]+1, dead_end[1]))
            if (dead_end[0]-1 >= 0) and (self.ship_display[dead_end[0]-1][dead_end[1]] == '■'):
                dead_end_blocked_neighbor.append((dead_end[0]-1, dead_end[1]))
            if (dead_end[1]+1 <= d-1) and (self.ship_display[dead_end[0]][dead_end[1]+1] == '■'):
                dead_end_blocked_neighbor.append((dead_end[0], dead_end[1]+1))
            if (dead_end[1]-1 >= 0) and (self.ship_display[dead_end[0]][dead_end[1]-1] == '■'):
                dead_end_blocked_neighbor.append((dead_end[0], dead_end[1]-1))
            #print(dead_end_blocked_neighbor)
            if dead_end_blocked_neighbor != []: #Make sure block neighbor was not previosuly removed
                random_blocked_neighbor = random.sample(dead_end_blocked_neighbor, 1)[0]
                self.ship_display[random_blocked_neighbor[0]][random_blocked_neighbor[1]] = 'O'
        
        self.open_ship = copy.deepcopy(self.ship_display) #Keep a copy of the original ship to see which cells are opened/blocked
    
    #Print a 2D array representing the ship in a way that makes it appear as a maze to a human. 
    #This version specifically only shows the original open and blocked cells when the maze is generated.
    def print_open_ship(self):
        for row in self.open_ship:
            formatted_row = ["{:1}".format(x) for x in row]
            print(" ".join(formatted_row))

    def flatten(self, row, col):
	    return row*self.dim + col



In [80]:
#Print a 2D array representing the ship in a way that makes it appear as a maze to a human.
#To be used for the ship display at specific times
def print_ship(ship):
    for row in ship.ship_display:
        formatted_row = ["{:1}".format(x) for x in row]
        print(" ".join(formatted_row))

In [81]:
#An example of the ship and it being printed two ways
ship_1 = ship(10)
ship_1.print_open_ship()
print()
print_ship(ship_1)

■ O O O ■ ■ O O O O
O O O O O ■ O O ■ O
O ■ ■ O ■ O ■ O O O
O O O O O O O ■ O ■
■ ■ ■ ■ O ■ O ■ O ■
■ O O O O O ■ O O O
O O ■ O ■ O O O ■ O
O ■ O O O ■ O O O O
O O O O O ■ O O ■ O
O O ■ O O O ■ O O O

■ O O O ■ ■ O O O O
O O O O O ■ O O ■ O
O ■ ■ O ■ O ■ O O O
O O O O O O O ■ O ■
■ ■ ■ ■ O ■ O ■ O ■
■ O O O O O ■ O O O
O O ■ O ■ O O O ■ O
O ■ O O O ■ O O O O
O O O O O ■ O O ■ O
O O ■ O O O ■ O O O


### 2,3,4 The Bot, The Fire, The Button

In [82]:
#Compute probability a cell catches fire
def prob_catching_fire(q, K):
    return 1 - (1-q)**K

#Place elements such as the fire starting position or the bot on the ship
def place_element(ship, element):
    random_cell_state = None
    while random_cell_state != 'O':
        row = random.randint(0, ship.dim-1)
        col = random.randint(0, ship.dim-1)
        if ship.open_ship[row][col] == 'O':
            ship.ship_display[row][col] = element
            random_cell_state = 'O'
            if element == 'B':
                ship.start = (row,col)
    return ship

In [83]:
ship_1 = place_element(ship_1, 'F')

In [84]:
#A function that takes some number of times to spread the fire, an already created ship object, and some flammability q
#Spreads the fire on the ship timestep many times and returns the updated ship
def spread_fire(timestep, ship, q):
    for t in range(timestep):
        ship_copy = copy.deepcopy(ship)
        for row in range(ship.dim):
            for col in range(ship.dim):
                if (ship.open_ship[row][col] == 'O') and (ship.ship_display[row][col] != 'F'):
                    neighbors_on_fire = 0
                    if (row+1 <= ship.dim-1) and (ship.ship_display[row+1][col] == 'F'):
                        neighbors_on_fire += 1
                    if (row-1 >= 0) and (ship.ship_display[row-1][col] == 'F'):
                        neighbors_on_fire += 1
                    if (col+1 <= ship.dim-1) and (ship.ship_display[row][col+1] == 'F'):
                        neighbors_on_fire += 1
                    if (col-1 >= 0) and (ship.ship_display[row][col-1] == 'F'):
                        neighbors_on_fire += 1
                    if random.uniform(0, 1) < prob_catching_fire(q, neighbors_on_fire):
                        ship_copy.ship_display[row][col] = 'F'
        ship = ship_copy
        # print_ship(ship)
        # print()

    return ship

In [85]:
#An exmaple of the fire spreading on a ship
ship_1 = spread_fire(10, ship_1, 0.5)
print_ship(ship_1)

■ O O O ■ ■ O O O O
O O O O O ■ O O ■ O
O ■ ■ F ■ F ■ O O O
O O F F F F F ■ O ■
■ ■ ■ ■ O ■ F ■ O ■
■ O O O O O ■ O O O
O O ■ O ■ O O O ■ O
O ■ O O O ■ O O O O
O O O O O ■ O O ■ O
O O ■ O O O ■ O O O


### 5 The Task

In [86]:
def bot1(ship):
	neighbors = []
	visited = set()
	prev = dict()
	neighbors.append(ship.start)
	visited.add(ship.start)
	prev[ship.start] = None
	board = ship.ship_display
	found = False
	while(True):
		if not neighbors:
			break
		row, col = neighbors.pop(0)
		if(ship.ship_display[row][col] == 'G'):
			return retrace(prev, (row, col))
			break
		
		if(row - 1 >= 0 and ship.ship_display[row-1][col] != '■' and ship.ship_display[row-1][col] != 'F' and (row-1, col) not in visited):
			neighbors.append((row-1, col))
			visited.add((row-1, col))
			prev[(row-1, col)] = (row, col)
		
		if(row + 1 < ship.dim and ship.ship_display[row+1][col] != '■' and ship.ship_display[row+1][col] != 'F' and (row+1, col) not in visited):
			neighbors.append((row+1, col))
			visited.add((row+1, col))
			prev[(row+1, col)] = (row, col)

		if(col-1 >= 0 and ship.ship_display[row][col-1] != '■' and ship.ship_display[row][col-1] != 'F' and (row, col-1) not in visited):
			neighbors.append((row, col-1))
			visited.add((row, col-1))
			prev[(row, col-1)] = (row, col)

		if(col + 1 < ship.dim and ship.ship_display[row][col+1] != '■' and ship.ship_display[row][col+1] != 'F' and (row, col+1) not in visited):
			neighbors.append((row, col+1))
			visited.add((row, col+1))
			prev[(row, col+1)] = (row, col)

def retrace(prev, start):
	route = []
	while(prev[start] != None):
		route.append(start)
		start = prev[start]
	route.reverse()	
	return route


In [87]:
d = 20
ship_2 = ship(d)

place_element(ship_2,'F')
place_element(ship_2,'B')
place_element(ship_2,'G')

print(ship_2.start)
print_ship(ship_2)
bot1(ship_2)

(9, 16)
O O O ■ O ■ O O O ■ O O O O ■ O ■ O O O
O ■ O O O O O ■ O ■ O ■ ■ O O O ■ O ■ O
O O O ■ O O O ■ O ■ O O O ■ O ■ O O ■ O
■ ■ O O O O O ■ O O ■ O ■ O O ■ O O O O
O O O O O ■ O O ■ O O O O O ■ O O O ■ O
O ■ O ■ O O ■ O ■ ■ ■ ■ O ■ O O O O O O
O ■ O ■ O ■ O O O O O O O O O ■ O ■ O O
O O O ■ O O O O O ■ O ■ O O ■ O O O ■ O
■ O ■ O O O ■ O ■ O O ■ O O ■ O ■ O O O
O O O ■ O ■ O O ■ O ■ O O O O O B ■ ■ O
O O O O O O O O O O O O ■ O ■ O ■ O ■ O
■ O ■ O ■ O G ■ O ■ ■ O ■ O ■ O O O O O
■ O ■ O ■ O ■ O O O O O O O O O ■ ■ O O
O O ■ ■ O O O O ■ O ■ O ■ O ■ O O O O ■
■ O O O O ■ ■ ■ O O O ■ O O ■ O ■ ■ O O
O ■ ■ ■ O ■ O O O O O O O O O O O O O O
O O O O ■ O O ■ O ■ ■ O ■ O ■ ■ O O O O
■ ■ ■ O O O ■ O O ■ ■ O F O O O O O O ■
O O O O O O O O O O O O ■ O ■ O O ■ O O
O O O ■ O O ■ O O ■ O ■ O O O O O O O O


[(9, 15),
 (9, 14),
 (9, 13),
 (9, 12),
 (9, 11),
 (10, 11),
 (10, 10),
 (10, 9),
 (10, 8),
 (10, 7),
 (10, 6),
 (11, 6)]

### 6 The Strategies

Bot 1

In [88]:
burnCount = 0
runs = 100
for k in range(runs):
    shipTest = ship(30)
    place_element(shipTest,'F')
    place_element(shipTest,'B')
    place_element(shipTest,'G')
    route = bot1(shipTest)
    if(not route): 
        burnCount += 1
    else:
        for i in range(len(route)):
            row, col = route[i]
            if shipTest.ship_display[row][col] == 'F':
                burnCount += 1
                break
            shipTest = spread_fire(1, shipTest, 0.5)

runs-burnCount

75

Bot 2

In [89]:
burnCount = 0
runs = 100
for k in range(runs):
    shipTest = ship(30)
    place_element(shipTest,'F')
    place_element(shipTest,'B')
    place_element(shipTest,'G')
    stop = False
    while(not stop): 
        route = bot1(shipTest)
        if(not route):
            burnCount += 1
            break
        else:
            row, col = route[0]
            shipTest.start = (row,col)
            if(shipTest.ship_display[row][col] == 'G'):
                break
            shipTest = spread_fire(1, shipTest, 0.5)
runs-burnCount

69

Bot 4

In [94]:
#Computes the probability that squares adjacent to the fire will be on fire in one timestep
#Computes the probability that squares two away from fire will be on fire in two timesteps
def prob_catching_fire_next_timestep(ship, q):
    #Keep a list of probabilities that a square will be on fire at the next timestep
    fire_probs = [[0 for i in range(d)] for j in range(d)]

    for row in range(ship.dim):
        for col in range(ship.dim):
            if (ship.open_ship[row][col] == 'O') and (ship.ship_display[row][col] != 'F'):
                neighbors_on_fire = 0
                if (row+1 <= ship.dim-1) and (ship.ship_display[row+1][col] == 'F'):
                    neighbors_on_fire += 1
                if (row-1 >= 0) and (ship.ship_display[row-1][col] == 'F'):
                    neighbors_on_fire += 1
                if (col+1 <= ship.dim-1) and (ship.ship_display[row][col+1] == 'F'):
                    neighbors_on_fire += 1
                if (col-1 >= 0) and (ship.ship_display[row][col-1] == 'F'):
                    neighbors_on_fire += 1

                fire_probs[row][col] = prob_catching_fire(q, neighbors_on_fire) #The probability of each square catching fire at the next time step
    
    #Keep a list of probabilities that a square will be on fire in two timesteps
    #Gets the expected value of number of neighbor cells on fire after one timestep to calculate the odds of a cell being on fire in two timesteps
    #See comments below for more details
    fire_probs_2 = [[0 for i in range(d)] for j in range(d)]
    for row in range(ship.dim):
        for col in range(ship.dim):
            if (ship.open_ship[row][col] == 'O') and (ship.ship_display[row][col] != 'F'):
                new_neighbors_on_fire = 0
                if (row+1 <= ship.dim-1):
                    if (ship.ship_display[row+1][col] == 'F'):
                        new_neighbors_on_fire += 1 #If the neighbor is on fire, you add 1 to the count of neighbors on fire
                    if (fire_probs[row+1][col] > 0 and fire_probs[row+1][col] < 1):
                        new_neighbors_on_fire += fire_probs[row+1][col] #There is a list of probabilities that a square will light on fire after one time step
                        #So you can no longer just add 1 if a square is on fire, because there is an X% chance the square is on fire
                        #Thus, find the expected value of the nighbor being on fire between 0 and 1, and add that to the number of neighbors on fire

                if (row-1 >= 0):
                    if (ship.ship_display[row-1][col] == 'F'):
                        new_neighbors_on_fire += 1
                    if (fire_probs[row-1][col] > 0 and fire_probs[row-1][col] < 1):
                        new_neighbors_on_fire += fire_probs[row-1][col]
                if (col+1 <= ship.dim-1):
                    if (ship.ship_display[row][col+1] == 'F'):
                        new_neighbors_on_fire += 1
                    if (fire_probs[row][col+1] > 0 and fire_probs[row][col+1] < 1):
                        new_neighbors_on_fire += fire_probs[row][col+1] 
                if (col-1 >= 0):
                    if (ship.ship_display[row][col-1] == 'F'):
                        new_neighbors_on_fire += 1
                    if (fire_probs[row][col-1] > 0 and fire_probs[row][col-1] < 1):
                        new_neighbors_on_fire += fire_probs[row][col-1] 

                fire_probs_2[row][col] = prob_catching_fire(q, new_neighbors_on_fire) #So now you have a list of probabilities of squares being on fire in two timesteps as well

    return fire_probs, fire_probs_2

In [95]:
#Largely the same as the code for bot1
#Modifications include calculating the probability that a square will catch fire in one and two timesteps (see cell above for specifics)
#This is done after every time step, similar to how bot 2 replans the route after every step

def bot4(ship, q):
	neighbors = []
	visited = set()
	prev = dict()
	neighbors.append(ship.start)
	visited.add(ship.start)
	prev[ship.start] = None
	board = ship.ship_display
	found = False
	while(True):
		if not neighbors:
			break
		row, col = neighbors.pop(0)
		if(ship.ship_display[row][col] == 'G'):
			return retrace(prev, (row, col))
			break

		p1,p2 = prob_catching_fire_next_timestep(ship, q)
		
		#Other modification is a probability threshold for the bot.
		#So if in one timestep, if the probability that a square will be on fire (calculated in cell above) is less than some percent (here 20), the bot will not go to that square
		#In two timesteps, the same metric is used for squares two away from the current fire, but the probability threshold is different (40)
		#So the bot does not strictly avoid cells adjacent to or twice removed from the fire. Rather it considers how likely the fire is to spread there and then decides whether or not to move
		if(row - 1 >= 0 and ship.ship_display[row-1][col] != '■' and ship.ship_display[row-1][col] != 'F' and (row-1, col) not in visited) and (p1[row-1][col] < 0.2) and (p2[row-1][col] < 0.4):
			neighbors.append((row-1, col))
			visited.add((row-1, col))
			prev[(row-1, col)] = (row, col)
		
		if(row + 1 < ship.dim and ship.ship_display[row+1][col] != '■' and ship.ship_display[row+1][col] != 'F' and (row+1, col) not in visited) and (p1[row+1][col] < 0.2) and (p2[row+1][col] < 0.4):
			neighbors.append((row+1, col))
			visited.add((row+1, col))
			prev[(row+1, col)] = (row, col)

		if(col-1 >= 0 and ship.ship_display[row][col-1] != '■' and ship.ship_display[row][col-1] != 'F' and (row, col-1) not in visited) and (p1[row][col-1] < 0.2) and (p2[row][col-1] < 0.4):
			neighbors.append((row, col-1))
			visited.add((row, col-1))
			prev[(row, col-1)] = (row, col)

		if(col + 1 < ship.dim and ship.ship_display[row][col+1] != '■' and ship.ship_display[row][col+1] != 'F' and (row, col+1) not in visited) and (p1[row][col+1] < 0.2) and (p2[row][col+1] < 0.4):
			neighbors.append((row, col+1))
			visited.add((row, col+1))
			prev[(row, col+1)] = (row, col)

def retrace(prev, start):
	route = []
	while(prev[start] != None):
		route.append(start)
		start = prev[start]
	route.reverse()	
	return route

In [92]:
#An example for bot4
d = 30
q = 0.5
ship_4 = ship(d)

place_element(ship_4,'F')
place_element(ship_4,'B')
place_element(ship_4,'G')

print(ship_4.start)
print_ship(ship_4)
bot4(ship_4, q)


(26, 23)
O O ■ O O ■ O ■ O O O O O O ■ O O O O F O O O O O O O ■ O O
■ O O O O O O O ■ O ■ ■ O O O O ■ O O ■ ■ O ■ O O ■ ■ O O O
O O O O ■ ■ O O O O ■ O O ■ ■ O ■ O ■ O O O O O O O O ■ O ■
O O ■ O O O O O ■ O O O ■ ■ ■ O ■ O ■ ■ O ■ O ■ O O O O O O
O O ■ ■ O ■ ■ O O ■ ■ ■ ■ ■ O O O O O O O O O O O O O ■ O O
O O O O ■ O O ■ O ■ O O ■ O O ■ ■ ■ ■ O ■ O O O ■ O O ■ O O
■ O ■ O O O ■ O O O ■ O O O ■ O O O O ■ O O ■ O ■ O O O O O
O O O ■ ■ O ■ ■ O ■ ■ ■ ■ O O O ■ O O O O O O O O O ■ ■ O O
O ■ O O O O O O O O ■ O O ■ O ■ O O O O O O O O O O O O O O
■ O O ■ O ■ O O O O O O O O O O O ■ O ■ ■ O O O O ■ ■ O ■ ■
O O O ■ O O O O O O O ■ ■ ■ ■ O ■ ■ O ■ O ■ O ■ ■ O O O O O
O ■ O O O ■ ■ ■ ■ O ■ O O O O O O ■ O O O O O O O O O O ■ O
O O ■ ■ O O O O O O O O O ■ O O O O O O O ■ O ■ ■ O ■ O O ■
■ O ■ ■ ■ O O O O O O ■ O O ■ O ■ O O O O O O ■ O O O O O O
O O O O ■ O ■ O O ■ O ■ ■ O O O O ■ ■ ■ O ■ O O O O ■ O O O
■ O ■ O ■ O O ■ ■ O O O O ■ O ■ O O O O O ■ O ■ O O O ■ O O
O O O O O ■ O ■ O ■ ■ ■ O O O G

[(25, 23),
 (25, 22),
 (24, 22),
 (24, 21),
 (23, 21),
 (23, 20),
 (22, 20),
 (21, 20),
 (20, 20),
 (19, 20),
 (19, 19),
 (18, 19),
 (17, 19),
 (17, 18),
 (17, 17),
 (17, 16),
 (17, 15),
 (16, 15)]

In [96]:
#Running trials for bot4
burnCount = 0
runs = 10
q=0.5
for k in range(runs):
    shipTest = ship(30)
    place_element(shipTest,'F')
    place_element(shipTest,'B')
    place_element(shipTest,'G')
    stop = False
    while(not stop): 
        route = bot4(shipTest, q)
        if(not route):
            burnCount += 1
            break
        else:
            row, col = route[0]
            shipTest.start = (row,col)
            if(shipTest.ship_display[row][col] == 'G'):
                break
            shipTest = spread_fire(1, shipTest, q)
runs-burnCount

7